In [1]:
from datetime import date

from create_data import recreate_databases, create_tables
from jobs.enrich_action import EnrichCEOrderCreatedJob
from utils import get_spark_session

In [2]:
spark = get_spark_session()

2021-11-26 16:37:59.316 | INFO     | utils:get_spark_session:7 - Creating Spark Session


In [3]:
recreate_databases(spark)

2021-11-26 16:38:08.646 | INFO     | create_data:recreate_databases:29 - Removing spark warehouse (path = 'spark-warehouse')
2021-11-26 16:38:08.698 | INFO     | create_data:recreate_databases:33 - Creating database 'standardized_glovo_live'
2021-11-26 16:38:11.164 | INFO     | create_data:recreate_databases:33 - Creating database 'mpcustomer_custom_events'
2021-11-26 16:38:11.210 | INFO     | create_data:recreate_databases:33 - Creating database 'mpcustomer_screen_views'
2021-11-26 16:38:11.254 | INFO     | create_data:recreate_databases:33 - Creating database 'enriched_custom_events'
2021-11-26 16:38:11.294 | INFO     | create_data:recreate_databases:33 - Creating database 'enriched_screen_views'


In [4]:
create_tables(spark)

2021-11-26 16:38:11.343 | INFO     | create_data:create_table:91 - Creating table 'standardized_glovo_live.cities'
2021-11-26 16:38:21.340 | INFO     | create_data:create_table:91 - Creating table 'standardized_glovo_live.devices'
2021-11-26 16:38:28.359 | INFO     | create_data:create_table:91 - Creating table 'standardized_glovo_live.orders'
2021-11-26 16:38:35.364 | INFO     | create_data:create_table:91 - Creating table 'standardized_glovo_live.customer_subscriptions'
2021-11-26 16:38:42.138 | INFO     | create_data:create_table:91 - Creating table 'standardized_glovo_live.customer_subscription_periods'
2021-11-26 16:38:48.674 | INFO     | create_data:create_table:91 - Creating table 'mpcustomer_custom_events.order_created'


In [5]:
order_created_job = EnrichCEOrderCreatedJob(
    spark=spark,
    exec_date=date(2021, 11, 19),
    n_days=3
)
order_created_job.run()

2021-11-26 16:38:55.874 | INFO     | loaders:load:47 - Loading 'mpcustomer_custom_events.order_created'
2021-11-26 16:38:56.040 | INFO     | loaders:load:47 - Loading 'standardized_glovo_live.cities'
2021-11-26 16:38:56.163 | INFO     | loaders:load:47 - Loading 'standardized_glovo_live.orders'
2021-11-26 16:38:56.201 | INFO     | ports.live_db:load:72 - Loading 'standardized_glovo_live.customer_subscriptions'
2021-11-26 16:38:56.377 | INFO     | transformations.interfaces:_apply:13 - Applying transformation 'AddTimezone'
2021-11-26 16:38:56.424 | INFO     | transformations.interfaces:_apply:13 - Applying transformation 'AddNumberOfOrders'
2021-11-26 16:38:56.894 | INFO     | transformations.interfaces:_apply:13 - Applying transformation 'AddIsPrime'
2021-11-26 16:38:57.039 | INFO     | transformations.interfaces:_apply:13 - Applying transformation 'AddLocalTime'
2021-11-26 16:38:57.167 | INFO     | writers:write:23 - Writting 'enriched_custom_events.order_created'


# Show input

In [6]:
spark.table("mpcustomer_custom_events.order_created").show()

+-----------------------+-----------+-------------------+---------------+
|custom_attributes__city|customer_id|      creation_time|p_creation_date|
+-----------------------+-----------+-------------------+---------------+
|                    BCN|          1|2021-11-19 12:00:00|     2021-11-19|
|                    BCN|          1|2021-11-18 08:00:00|     2021-11-18|
|                    CAG|          2|2021-11-17 05:05:00|     2021-11-17|
|                    CAG|          2|2021-11-01 07:29:00|     2021-11-01|
+-----------------------+-----------+-------------------+---------------+



# Show outputs

In [7]:
spark.table("enriched_custom_events.order_created").show()

+-----------------------+-----------+-------------------+---------------+-------------+------------+----------------+--------+-------------------+-----------+
|custom_attributes__city|customer_id|      creation_time|p_creation_date|    time_zone|country_code|number_of_orders|is_prime|creation_time_local|time_of_day|
+-----------------------+-----------+-------------------+---------------+-------------+------------+----------------+--------+-------------------+-----------+
|                    BCN|          1|2021-11-19 12:00:00|     2021-11-19|Europe/Madrid|          ES|               2|   false|2021-11-19 13:00:00|      Lunch|
|                    BCN|          1|2021-11-18 08:00:00|     2021-11-18|Europe/Madrid|          ES|               2|   false|2021-11-18 09:00:00|  Breakfast|
|                    CAG|          2|2021-11-17 05:05:00|     2021-11-17|  Europe/Rome|          IT|               1|    true|2021-11-17 06:05:00|  Breakfast|
+-----------------------+-----------+---------